```
conda create -n hw10 python=3.11 -y

conda activate hw10

pip3 install torch 

conda install numpy matplotlib pandas scipy jupyter notebook ipykernel -y

pip install tqdm transformers

python -m ipykernel install --user --name hw10 --display-name "Python 3.11 (hw10)"
```

In [1]:
import os
import re
import warnings
import random
from collections import defaultdict
from typing import Dict, List, Tuple  # намёк на использование)

import torch
import torch.nn.functional as F
import numpy as np
import numpy as np
import torch
from tqdm.notebook import tqdm
from transformers import GPT2LMHeadModel, GPT2Tokenizer

warnings.filterwarnings("ignore")

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

## Задание

1) Реализовать методы `greedy_sampling` и `generate` (1 балл)
2) Реализовать метод `random_sampling` и поддержать его в `generate` (1 балл)
3) Реализовать метод `_beam_search_generate` и поддержать его в `generate` (2 балла)
4) Реализовать методы `apply_top_p`, `apply_top_k`, `apply_temperature` и поддержать их в `generate` (1 балл)  
Все методы необходимо реализовать через векторные операции в torch/numpy везде где это возможно

### __Greedy Sampling (Жадный выбор)__
Принцип работы:<br>
На каждом шаге выбирается самый вероятный токен (слово)

Плюсы:
- Быстрый и простой
- Детерминированный 

Минусы:
- Предсказуемый и скучный текст
- Может застрять в повторениях 

## __Random Sampling__
Принцип работы:<br>
Выбирается токен случайно, согласно распределению вероятностей модели


Плюсы:
- Разнообразный текст
- Креативные результаты

Минусы:
- Может генерировать бессмыслицу
- Недетерминированный (каждый раз разный результат)
- Может выбрать очень маловероятные слова

Улучшения: Temperature, Top-K, Top-P (об этом позже)

### __Beam Search__
Принцип работы:<br>
Держим несколько гипотез (лучей) одновременно и выбираем лучшую последовательность целиком.

Плюсы:
- Находит более оптимальные последовательности
- Хорош для задач с умным / правильным ответом (перевод, суммаризация)
- Детерминированный

Минусы:
- Медленнее в beam_size раз
- Требует больше памяти

In [ ]:
class Model:
    def __init__(self, model_name: str = "gpt2"):
        self.model = GPT2LMHeadModel.from_pretrained(model_name)
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.vocab_size = self.tokenizer.vocab_size



    def greedy_sampling(self, logits: torch.Tensor) -> int:
        return torch.argmax(logits, dim=-1).item()
    


    def random_sampling(self, logits: torch.Tensor) -> int:
        # Преобразуем logits в вероятности через softmax
        probs = F.softmax(logits, dim=-1)

        # Случайно выбираем токен согласно вероятностям
        next_token = torch.multinomial(probs, num_samples=1)
    
        return next_token.item()
    


    def _beam_search_generate(
        self,
        prompt: str,
        max_length: int,
        num_beams: int
    ) -> str:
        # Токенизируем промпт
        input_ids = self.tokenizer.encode(prompt, return_tensors="pt")

        # Инициализацию начинаем с одного луча
        # beams хранит список кортежей: (последовательность_токенов, накопленный_score)
        beams = [(input_ids, 0.0)]  # score в log-пространстве (начинаем с log(1) = 0)

        # Генерируем токены до достижения max_length
        for _ in range(max_length):
            all_candidates = []

            # Для каждого луча генерируем продолжения
            for seq, score in beams:
                # Получаем предсказания модели
                with torch.no_grad():
                    outputs = self.model(seq)
                    logits = outputs.logits[0, -1, :]  # берём logits последнего токена

                # Преобразуем в log-вероятности
                log_probs = F.log_softmax(logits, dim=-1)

                # Берём top-k токенов с наибольшими вероятностями
                topk_log_probs, topk_indices = torch.topk(log_probs, num_beams)

                # Создаём новые кандидаты
                for i in range(num_beams):
                    next_token = topk_indices[i].unsqueeze(0).unsqueeze(0)
                    new_seq = torch.cat([seq, next_token], dim=-1)
                    new_score = score + topk_log_probs[i].item()

                    all_candidates.append((new_seq, new_score))

            # Сортируем всех кандидатов по score и берём лучшие num_beams
            beams = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:num_beams]

            # Проверяем, все ли лучи закончились (встретили EOS токен)
            if all(seq[0, -1].item() == self.tokenizer.eos_token_id for seq, _ in beams):
                break
            
        # Возвращаем лучшую гипотезу
        best_seq = beams[0][0]
        return self.tokenizer.decode(best_seq[0], skip_special_tokens=True)
    


    def apply_temperature(self, logits: torch.Tensor, temperature: float = 1.0) -> torch.Tensor:
        return logits / temperature

    def _apply_top_k(self, logits: torch.Tensor, top_k: int = 0) -> torch.Tensor:  # тут наверное int
        if top_k <= 0:
            return logits

        # Получаем top_k значений
        top_k = min(top_k, logits.size(-1))
        topk_values, _ = torch.topk(logits, top_k)
        min_value = topk_values[..., -1, None]

        # Обнуляем все значения меньше минимального из top_k
        return torch.where(
            logits < min_value,
            torch.full_like(logits, float('-inf')),
            logits
        )

    def _apply_top_p(self, logits: torch.Tensor, top_p: float = 1) -> torch.Tensor:  # self перенести в функцию
        if top_p >= 1.0:
            return logits
        
        # Сортируем logits по убыванию
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        
        # Преобразуем в вероятности
        sorted_probs = F.softmax(sorted_logits, dim=-1)
        
        # Считаем кумулятивную сумму вероятностей
        cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
        
        # Находим токены, которые нужно удалить (те, что после порога top_p)
        # Сдвигаем на 1 вправо, чтобы включить первый токен, который превысил порог
        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = False
        
        # Создаём маску для исходных индексов
        indices_to_remove = torch.zeros_like(logits, dtype=torch.bool)
        indices_to_remove[sorted_indices] = sorted_indices_to_remove
        
        # Обнуляем отфильтрованные токены
        logits = logits.masked_fill(indices_to_remove, float('-inf'))
        
        return logits

    def generate(
        self,
        prompt: str,
        max_length: int = 50,
        strategy: str = "greedy",
        temperature: float = 1.0,
        top_k: int = 0,
        top_p: float = 1.0,
        num_beams: int = 3
    ) -> str:
        # Beam search обрабатывается отдельно
        if strategy == "beam":
            return self._beam_search_generate(prompt, max_length, num_beams)
        
        # Для greedy и random используем autoregressive generation
        input_ids = self.tokenizer.encode(prompt, return_tensors="pt")
        
        for _ in range(max_length):
            # Получаем предсказания модели
            with torch.no_grad():
                outputs = self.model(input_ids)
                logits = outputs.logits[0, -1, :]  # logits последнего токена
            
            # Применяем фильтрацию (только для random sampling)
            if strategy == "random":
                logits = self.apply_temperature(logits, temperature)
                logits = self._apply_top_k(logits, top_k)
                logits = self._apply_top_p(logits, top_p)
            
            # Выбираем следующий токен
            if strategy == "greedy":
                next_token = self.greedy_sampling(logits)
            elif strategy == "random":
                next_token = self.random_sampling(logits)
            else:
                raise ValueError(f"Unknown strategy: {strategy}")
            
            # Добавляем токен к последовательности
            input_ids = torch.cat([
                input_ids,
                torch.tensor([[next_token]])
            ], dim=-1)
            
            # Останавливаемся при встрече EOS токена
            if next_token == self.tokenizer.eos_token_id:
                break
            
        # Декодируем результат
        return self.tokenizer.decode(input_ids[0], skip_special_tokens=True)

In [4]:
# Продемонстрируйте результат работы `generate` при различных параметрах

model = Model("gpt2")

prompt = "how are you? what do you want?"


# Greedy Sampling
print("\n1. greedy sampling")

result = model.generate(prompt, max_length=30, strategy="greedy")
print(result)

# Random Sampling (может быть хаотичным)
print("\n2. Random Sampling")

result = model.generate(prompt, max_length=30, strategy="random")
print(result)

# Random + Temperature = 0.7 (более консервативный)
print("\n3. RRandom + Temperature = 0.7 (более уверенный)")

result = model.generate(prompt, max_length=30, strategy="random", temperature=0.7)
print(result)

# Random + Temperature = 1.5 (более креативный)
print("\n4. Random + Temperature = 1.5 (более случайный)")

result = model.generate(prompt, max_length=30, strategy="random", temperature=1.5)
print(result)

# 5. Random + Top-K = 50
print("\n5. Random + Top-K = 50")

result = model.generate(prompt, max_length=30, strategy="random", top_k=50)
print(result)

# 6. Random + Top-P = 0.9 
print("\n6. Random + Top-P = 0.9")

result = model.generate(prompt, max_length=30, strategy="random", top_p=0.9)
print(result)

# Комбинация всех 
print("\n7. КОМБО: Temperature=0.8 + Top-K=40 + Top-P=0.95")

result = model.generate(
    prompt, 
    max_length=30, 
    strategy="random", 
    temperature=0.8,
    top_k=40,
    top_p=0.95
)
print(result)

# Beam Search с 3 последовательностями
print("\n8. Beam Search (num_beams=3)")

result = model.generate(prompt, max_length=30, strategy="beam", num_beams=3)
print(result)

# Beam Search с 5 последовательностями
print("\n9. Beam Search (num_beams=5)")

result = model.generate(prompt, max_length=30, strategy="beam", num_beams=5)
print(result)


1. greedy sampling
how are you? what do you want? what do you want? what do you want? what do you want? what do you want? what do you want? what do you want?

2. Random Sampling
how are you? what do you want? I'm sorry you guessed I wasn't more specific than what i am like then again would we say it's for siblings)."

WolfLur

3. RRandom + Temperature = 0.7 (более уверенный)
how are you? what do you want? what do you want?

The answer is: We are men.

Women are beautiful and we can have sex with them. Men are

4. Random + Temperature = 1.5 (более случайный)
how are you? what do you want? thinkvm unit Mark recognizes W Speech gniaz proposalsy fight news Aldvin Ret Console TABLE Eyegarden Tempest rearrononde es prisoFG 46

5. Random + Top-K = 50
how are you? what do you want? what is your relationship with the man?"

Barry's response came off kind of boring, with a lot of awkward questions like, "what

6. Random + Top-P = 0.9
how are you? what do you want?

Something that we've worked o

надо было сохранять первые выводы, пока не нашёл все ошибки в реализации)

правилом 3ёх H и не пахло

```
and we can have sex with them

Men are
```
смешно

# Выводы

## 1. **Greedy Sampling** - проблема зацикливания 
Застрял в бесконечном повторении - выбирает локально оптимальное, но глобально плохое решение



## 2. **Random Sampling** 
Генерирует разнообразный, но грамматически сомнительный текст. Может выбирать маловероятные слова



## 3. **Temperature = 0.7** - неожиданный результат 


"Women are beautiful and we can have sex with them"

а как же правило трёх H

 недостаточно консервативен 





## 4. **Temperature = 1.5** - полный бред 




## 5. **Top-K = 50** - тоже не очень адекватно в конце